<a href="https://colab.research.google.com/github/tanvircr7/meh/blob/master/PrisonnersDilemma_Langgraph.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
! pip install langchain_community tiktoken langchain-openai langchainhub chromadb langchain langgraph tavily-python

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.3/67.3 kB 4.2 MB/s eta 0:00:00
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 43.5/43.5 kB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.5/2.5 MB 34.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 62.9/62.9 kB 4.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 18.9/18.9 MB 91.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 94.9/94.9 kB 7.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 155.3/155.3 kB 7.8 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 284.2/284.2 kB 21.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 76.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 44.2/44.2 kB 3.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 48.9/48.9 kB 4.0 MB/s eta 0:00:00

In [6]:

import langgraph.graph as lg
from langgraph.graph import StateGraph, START, END
from langchain_openai import ChatOpenAI

# Initialize LLM (Optional for richer decision-making if you integrate strategies with LLMs)
# eg., llm = ChatOpenAI(model="gpt-4")

# Define the game state
from typing import List

from typing_extensions import TypedDict

class GameState():
    agents = {"agent1": {"payoff": 0}, "agent2": {"payoff": 0}}
    turn = 0
    max_turns = 5  # Set how many rounds the game should last
    actions_log = []  # Log decisions for analysis

# Define the actions and payoff matrix
payoff_matrix = {
    ("C", "C"): (3, 3),  # Both cooperate
    ("C", "D"): (0, 5),  # Agent1 cooperates, Agent2 defects
    ("D", "C"): (5, 0),  # Agent1 defects, Agent2 cooperates
    ("D", "D"): (1, 1),  # Both defect
}

# Simple strategy for agents: random, always cooperate, or tit-for-tat
def agent_strategy(state, agent_id):
    current_turn = state.turn
    # Example strategy: Tit-for-Tat (mirror last opponent move)
    if current_turn == 0:
        # Cooperate in the first round
        return "C"
    else:
        opponent_id = "agent1" if agent_id == "agent2" else "agent2"
        # Mirror last opponent action
        last_actions = state.actions_log[-1]  # Get last round's actions
        return last_actions[opponent_id]

# Environment node: resolve actions and update payoffs
def environment_update(state):
    # Get agent actions from the current turn
    game_actions = {
        agent_id: agent_strategy(state, agent_id)
        for agent_id in state.agents
    }
    state.actions_log.append(game_actions)

    # Extract actions for this round
    action1, action2 = game_actions["agent1"], game_actions["agent2"]

    # Compute payoffs based on the payoff matrix
    payoff1, payoff2 = payoff_matrix[(action1, action2)]
    state.agents["agent1"]["payoff"] += payoff1
    state.agents["agent2"]["payoff"] += payoff2

    # Increment the turn
    state.turn += 1

    # Print game progress (for visualization/debugging)
    print(f"Turn {state.turn}:")
    print(f"Agent1 chose {action1}, Agent2 chose {action2}")
    print(f"Payoffs -> Agent1: {state.agents['agent1']['payoff']}, Agent2: {state.agents['agent2']['payoff']}\n")

    return state

# Game termination condition
def end_game_condition(state):
    if state.turn < state.max_turns:
      return "agent1"
    return "END"

# Building the LangGraph for the game
game_graph = StateGraph(GameState)

# Add nodes for agent decisions and environment updates
game_graph.add_node("agent1", lambda state: agent_strategy(state, "agent1"))
game_graph.add_node("agent2", lambda state: agent_strategy(state, "agent2"))
game_graph.add_node("environment", environment_update)

# Define how the graph flows (edges between nodes)
game_graph.add_edge(START, "agent1")
game_graph.add_edge("agent1", "agent2")  # Agent1 makes decision, then Agent2
game_graph.add_edge("agent2", "environment")  # Agent2 makes decision, then update environment

# Add conditional edges to loop until game ends
game_graph.add_conditional_edges(
    "environment",
    end_game_condition,
    {"agent1": "agent1", "END": END}  # Loop back to Agent1 or terminate game
)

# Compile and run the game
prisoners_dilemma_game = game_graph.compile()

# Run the game simulation
# Run the game simulation
# Create an initial GameState instance and pass it to the invoke method
initial_state = GameState()
prisoners_dilemma_game.invoke(initial_state)



InvalidUpdateError: Expected dict, got <__main__.GameState object at 0x78118965d110>
For troubleshooting, visit: https://python.langchain.com/docs/troubleshooting/errors/INVALID_GRAPH_NODE_RETURN_VALUE